<a href="https://colab.research.google.com/github/auroraji2906/nlp/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
git clone https://github.com/amazon-science/SigExt.git
cd SigExt

Cloning into 'SigExt'...


In [2]:
# Librerie principali
!pip install torch torchvision torchaudio --quiet
!pip install transformers datasets --quiet
!pip install pytorch-lightning --quiet
!pip install jsonlines rake_nltk tqdm numpy pandas scikit-learn --quiet
!pip install rapidfuzz rouge-score matplotlib seaborn --quiet

# NLTK setup
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 46.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.5 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
%cd /content/SigExt

/content/SigExt


#AESLC

In [4]:
%%bash
python3 src/prepare_data.py --dataset aeslc --output_dir experiments/aeslc_dataset/

100%|██████████| 500/500 [00:04<00:00, 124.37it/s]


In [5]:
!python3 src/train_longformer_extractor_context.py \
  --dataset_dir experiments/aeslc_dataset/ \
  --checkpoint_dir experiments/aeslc_extractor_model/

Seed set to 42
config.json: 100% 694/694 [00:00<00:00, 5.37MB/s]
vocab.json: 899kB [00:00, 32.4MB/s]
merges.txt: 456kB [00:00, 93.7MB/s]
tokenizer.json: 1.36MB [00:00, 88.2MB/s]
INFO:absl:Using default tokenizer.
process data: 100% 1000/1000 [00:09<00:00, 109.76it/s]
INFO:root:keyword ratio 0.06956394146757906
INFO:root:Dataset size: 714
INFO:absl:Using default tokenizer.
process data: 100% 200/200 [00:02<00:00, 71.15it/s] 
INFO:root:keyword ratio 0.10042796005706134
INFO:root:Dataset size: 44
2025-12-12 12:21:27.475925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765542087.492796    1312 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765542087.497764    1312 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register f

In [6]:
!python3 src/inference_longformer_extractor.py \
  --dataset_dir experiments/aeslc_dataset/ \
  --checkpoint_dir experiments/aeslc_extractor_model/ \
  --output_dir experiments/aeslc_dataset_with_keyphrase/

INFO:root:Using fexperiments/aeslc_extractor_model/lightning_logs/version_0/checkpoints/epoch_01-step_001428-recall20_0.905.ckpt
2025-12-12 12:28:44.842239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765542524.859373    3206 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765542524.864296    3206 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765542524.876765    3206 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765542524.876788    3206 computation_placer.cc:177] computation placer already registered. Please check linkage and avoi

In [7]:
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 137.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [8]:
from huggingface_hub import login

login()  # ti chiederà il token


In [9]:
!python3 src/zs_summarization.py \
  --model_name llama \
  --kw_strategy sigext_topk \
  --kw_model_top_k 15 \
  --dataset aeslc \
  --dataset_dir experiments/aeslc_dataset_with_keyphrase/ \
  --output_dir experiments/aeslc_extsig_predictions/


tokenizer_config.json: 100% 54.5k/54.5k [00:00<00:00, 10.1MB/s]
tokenizer.json: 100% 9.09M/9.09M [00:00<00:00, 62.2MB/s]
special_tokens_map.json: 100% 296/296 [00:00<00:00, 2.64MB/s]
config.json: 100% 878/878 [00:00<00:00, 7.59MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-12 12:52:23.834587: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765543943.855981    9082 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765543943.861308    9082 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765543943.877264    9082 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same ta